In [0]:
%tensorflow_version 2.x

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras as k

from tensorflow.keras import layers as l

from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_csv('/content/drive/My Drive/procSelectCharLines.csv')

In [0]:
df.head()

,Season,Episode,Character,Line,Processed Line
0,10,1,Stan,"You guys, you guys! Chef is going away.",you guys you guys chef is going away
1,10,1,Kyle,Going away? For how long?,going away for how long
2,10,1,Stan,"Chef said he's been bored, so he joining a gro...",chef said hes been bored so he joining a group...
3,10,1,Cartman,I'm gonna miss him. I'm gonna miss Chef and I...,im gonna miss him im gonna miss chef and i an...
4,10,1,Stan,"Dude, how are we gonna go on? Chef was our fuh...",dude how are we gonna go on chef was our fuh f...


In [0]:
def map_char(char):
  for i, chars in enumerate(df['Character'].unique()):
    if char == chars:
      return i
  
  return None

In [0]:
df['Target'] = df['Character'].map(map_char)

In [0]:
df.head()

,Season,Episode,Character,Line,Processed Line,Target
0,10,1,Stan,"You guys, you guys! Chef is going away.",you guys you guys chef is going away,0
1,10,1,Kyle,Going away? For how long?,going away for how long,1
2,10,1,Stan,"Chef said he's been bored, so he joining a gro...",chef said hes been bored so he joining a group...,0
3,10,1,Cartman,I'm gonna miss him. I'm gonna miss Chef and I...,im gonna miss him im gonna miss chef and i an...,2
4,10,1,Stan,"Dude, how are we gonna go on? Chef was our fuh...",dude how are we gonna go on chef was our fuh f...,0


In [0]:
#!wget http://nlp.stanford.edu/data/glove.6B.zip

In [0]:
# download pretrained GloVe embeddings
#!wget "https://nlp.stanford.edu/data/glove.840B.300d.zip"

--2020-04-06 13:39:51--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
Unable to establish SSL connection.


In [0]:
RANDOM = 1
lines, targets = [], []

for line, targ in zip(df['Processed Line'], df['Target']):
  lines.append(line)
  targets.append(targ)

x_train, x_test, y_train,  y_test = train_test_split(lines, targets, 
                                                    test_size=0.1, 
                                                    random_state=RANDOM)

y_train = np.array(y_train)
y_test = np.array(y_test)

In [0]:
words = []

for line in df['Processed Line']:
  tokens = line.split()
  words.extend([word for word in tokens])

vocab = sorted(set(words))

In [0]:
len(vocab)

14636

In [0]:
vocab_size = 14500
embed_dim = 16
max_len = 50
trunc = 'post'
pad = 'post'
oov = '<OOV>'

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size,
                                                          oov_token = oov)

tokenizer.fit_on_texts(x_train)

word2idx = tokenizer.word_index

train_seq = tokenizer.texts_to_sequences(x_train)
test_seq = tokenizer.texts_to_sequences(x_test)

train_pad = tf.keras.preprocessing.sequence.pad_sequences(
    train_seq, padding=pad, maxlen=max_len, truncating=trunc
)
test_pad = tf.keras.preprocessing.sequence.pad_sequences(
    test_seq, maxlen=max_len
)


In [0]:
idx2word = dict([(v,k) for (k,v) in word2idx.items()])

## Basic Model - Initial baseline

In [0]:
def build_model(units = 32, layer_type = 'lstm', bidirectional = False, rec_count = 1):
  
  # units - units in recurrent layers
  # layer_type - one of {'lstm', 'gru', 'conv'}
  # bidirectional - bidirectional recurrent layers
  # rec_count - number of recurrent layers

  layer_types = ['lstm', 'gru', 'conv']

  if layer_type not in layer_types:
    raise ValueError("Invalid layer type. Expected one of: {}".format(repr(layer_types)))
  
  model = k.Sequential()
  # Text modelling begins with embeddings
  model.add(k.layers.Embedding(vocab_size, embed_dim, input_length=max_len))

  if layer_type == 'lstm':

    if bidirectional:
      for _ in range(rec_count-1):
        # Recurrent layers which pass their outputs to another recurrent layer 
        # need the `return_sequences` argument set to True.
        model.add(k.layers.Bidirectional(k.layers.LSTM(units, return_sequences=True, activation='relu')))
      # Final recurrent layer does not need to return sequences
      model.add(k.layers.Bidirectional(k.layers.LSTM(units, activation='relu')))

    else:
      for _ in range(rec_count-1):
        model.add(k.layers.LSTM(units, activation='relu', return_sequences=True))
      model.add(k.layers.LSTM(units, activation='relu'))

  elif layer_type == 'gru':

    if bidirectional:
      for _ in range(rec_count-1):
        model.add(k.layers.Bidirectional(k.layers.GRU(units, activation='relu', return_sequences=True)))
      model.add(k.layers.Bidirectional(k.layers.GRU(units, activation='relu')))

    else:
      for _ in range(rec_count-1):
        model.add(k.layers.GRU(units, activation='relu', return_sequences=True))
      model.add(k.layers.GRU(units, activation='relu'))

  else:

    model.add(k.layers.Conv1D(128, 5, activation='relu'))
    
    # Either average or max pooling may be used - you could even use both
    # if using the functional API
    model.add(k.layers.GlobalAveragePooling1D())
    #model.add(k.layers.GlobalMaxPooling1D())

  # Dense layers form the head of our network as usual
  model.add(k.layers.Dense(32, activation='relu'))
  model.add(k.layers.Dense(6, activation='sigmoid'))

  return model

In [0]:
model = build_model(32, 'conv', True, 2)

In [0]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [0]:
epochs = 10
model.fit(train_pad, y_train, epochs=epochs, validation_data=(test_pad, y_test))

Epoch 1/10
665/665 [==============================] - 6s 10ms/step - loss: 1.5639 - accuracy: 0.3349 - val_loss: 1.4968 - val_accuracy: 0.3506
Epoch 2/10
665/665 [==============================] - 6s 9ms/step - loss: 1.4121 - accuracy: 0.3736 - val_loss: 1.4049 - val_accuracy: 0.3980
Epoch 3/10
665/665 [==============================] - 6s 9ms/step - loss: 1.2422 - accuracy: 0.4485 - val_loss: 1.3849 - val_accuracy: 0.4174
Epoch 4/10
665/665 [==============================] - 6s 9ms/step - loss: 1.1318 - accuracy: 0.4960 - val_loss: 1.4579 - val_accuracy: 0.3920
Epoch 5/10
665/665 [==============================] - 6s 10ms/step - loss: 1.0603 - accuracy: 0.5258 - val_loss: 1.4879 - val_accuracy: 0.3908
Epoch 6/10
665/665 [==============================] - 6s 10ms/step - loss: 1.0039 - accuracy: 0.5592 - val_loss: 1.5697 - val_accuracy: 0.3895
Epoch 7/10
665/665 [==============================] - 6s 10ms/step - loss: 0.9575 - accuracy: 0.5759 - val_loss: 1.5705 - val_accuracy: 0.3891
Ep

Practice BERT